# Добро пожаловать в игру "Школа 2025"

Ниже представлен класс инициализации для игры, который затем будет использоваться при прохождении

In [11]:
class GameEngine:
    def __init__(self):
        self.status = 0
        self.scenes = {
                        "Start scene": 
                            { 
                            "First choice": "First consequence",
                            "Second choice": "Second consequence",
                            "Third choice": "Third consequence"
                            },
                        "First consequence":
                            {
                            "First.First choice": "Early end 1",
                            "First.Second choice": "Fourth consequence"
                            },
                        "Second consequence": 
                            {
                            "Second.First choice": "Early end 2",
                            "Second.Second choice": "Second consequence"
                            },
                        "Third consequence":
                            {
                            "Third.First choice": "Early end 3",
                            "Third.Second choice": "Early end 4"
                            },
                        "Fourth consequence": 
                            {
                            "Fourth.First choice": "Early end 5",
                            "Fourth.Second choice": "Early end 6"
                            }
                
        self.counter = 0

        def dfs(self):
            pass

        def check_status(self):
            match self.status:
                case 0:
                    start()
                case 1:
                    first_scene()
                case 2:
                    second_scene()
                case 3:
                    third_scene()
                case 4:
                    fourth_scene()
                
                    